In [ ]:
import pandas as pd
import numpy as np  # Importing numpy

def merge_all_data(user_health_data_file, supplement_usage_file, experiments_file, user_profiles_file):
    # Read the data from CSV files
    user_health_data = pd.read_csv(user_health_data_file)
    supplement_usage = pd.read_csv(supplement_usage_file)
    experiments = pd.read_csv(experiments_file)
    user_profiles = pd.read_csv(user_profiles_file)

    # Strip any leading/trailing spaces from the column names
    user_health_data.columns = user_health_data.columns.str.strip()
    supplement_usage.columns = supplement_usage.columns.str.strip()
    experiments.columns = experiments.columns.str.strip()
    user_profiles.columns = user_profiles.columns.str.strip()

    # Check if 'user_id' and 'date' columns are present in user_health_data
    if 'user_id' not in user_health_data.columns or 'date' not in user_health_data.columns:
        raise KeyError("Missing required columns in user_health_data. Expected 'user_id' and 'date'.")

    # Convert 'dosage' in supplement_usage to grams if it's in mg
    supplement_usage['dosage_grams'] = supplement_usage.apply(
        lambda row: row['dosage'] / 1000 if row['dosage_unit'] == 'mg' else row['dosage'],
        axis=1
    )

    # Merge supplement_usage with experiments to get the experiment name
    supplement_experiment = pd.merge(supplement_usage, experiments[['experiment_id', 'name']], on='experiment_id', how='left')

    # Rename 'name' column to 'experiment_name'
    supplement_experiment.rename(columns={'name': 'experiment_name'}, inplace=True)

    # Merge user_health_data with supplement_experiment based on 'user_id' and 'date'
    combined_data = pd.merge(user_health_data, supplement_experiment, on=['user_id', 'date'], how='outer')

    # Merge combined_data with user_profiles to get user information (email and age)
    final_data = pd.merge(combined_data, user_profiles[['user_id', 'email', 'age']], on='user_id', how='left')

    # **Type Conversion**, adjusting data types as per requirements
    final_data['email'] = final_data['email'].astype('str')  # Convert to string
    final_data['is_placebo'] = final_data['is_placebo'].astype('bool')  # Convert to boolean
    
    # Convert 'date' to datetime64 type
    final_data['date'] = pd.to_datetime(final_data['date'], errors='coerce')  # Convert to datetime64

    # Ensure 'date' is in the correct format (without time)
    # final_data['date'] = final_data['date'].dt.date  # Convert to date (remove time part)

    # **Handle missing values**: fill missing values according to the description
    final_data['supplement_name'] = final_data['supplement_name'].fillna('No intake')  # 'No intake' for days without supplement
    final_data['dosage_grams'] = final_data['dosage_grams'].fillna(np.nan)  # Fill missing dosage values with np.nan
    final_data['is_placebo'] = final_data['is_placebo'].fillna(np.nan)  # Fill missing placebo info with np.nan
    final_data['experiment_name'] = final_data['experiment_name'].fillna(np.nan)  # Fill missing experiment names with np.nan

    # Handle 'user_age_group' based on age
    def get_age_group(age):
        if pd.isna(age):
            return 'Unknown'
        if age < 18:
            return 'Under 18'
        elif 18 <= age <= 25:
            return '18-25'
        elif 26 <= age <= 35:
            return '26-35'
        elif 36 <= age <= 45:
            return '36-45'
        elif 46 <= age <= 55:
            return '46-55'
        elif 56 <= age <= 65:
            return '56-65'
        else:
            return 'Over 65'
    
    final_data['user_age_group'] = final_data['age'].apply(get_age_group)

    # Ensure no missing values for 'user_id' and 'date' columns
    if final_data[['user_id', 'date']].isnull().any().any():
        raise ValueError("Missing values in user_id or date columns in the final data.")

    # Select final columns based on the required output
    final_columns = [
        'user_id', 'date', 'email', 'user_age_group', 'experiment_name',
        'supplement_name', 'dosage_grams', 'is_placebo', 'average_heart_rate',
        'average_glucose', 'sleep_hours', 'activity_level'
    ]
    final_data = final_data[final_columns]

    # Clean the 'sleep_hours' column by removing 'h' or 'H' and converting it to float
    final_data['sleep_hours'] = final_data['sleep_hours'].replace({'h': '', 'H': ''}, regex=True).astype(float)

    # Remove duplicate rows
    final_data = final_data.drop_duplicates()

    # Return the final merged DataFrame
    return final_data


